<a href="https://colab.research.google.com/github/josh-ashkinaze/Clustering-Democrats/blob/master/WorkingChains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [1]:
!pip install openai
import numpy as np
from openai import OpenAI
import random

In [10]:
my_key = "asd"

In [32]:
import numpy as np
from openai import OpenAI
import random
import pandas as pd
from datetime import datetime
client = OpenAI(api_key=my_key)

anes_mapping = {

    "birthyr_dropdown": {
    "name": "Your age is",
    "values": {
      "integer": ""
    }
  },

  "educ": {
    "name": "Your education is",
    "values": {
      "1": "No high school",
      "2": "High school graduate",
      "3": "Some college",
      "4": "College graduate"
    }
  },
  "gender": {
    "name": "Your gender is",
    "values": {
      "1": "Male",
      "2": "Female"
    }
  },
  "race": {
    "name": "Your race is",
    "values": {
      "eth": {
        "1": "Hispanic",
        "2": ""
      },
      "rwh": {
        "1": "White",
        "2": ""
      },
      "rbl": {
        "1": "Black",
        "2": ""
      },
      "rain": {
        "1": "American Indian or Alaska Native",
        "2": ""
      },
      "ras": {
        "1": "Asian",
        "2": ""
      },
      "rpi": {
        "1": "Native Hawaiian or Pacific Islander",
        "2": ""
      },
      "roth": {
        "1": "Other",
        "2": ""
      }
    }
  },

      "pid3": {
    "name": "Politically, you identify as a(n)",
    "values": {
      "-7": "No Answer",
      "1": "Democrat",
      "2": "Republican",
      "3": "Independent",
      "4": "Other",
      "5": "Not sure"
    }
  },
  "ideo5": {
    "name": "Your ideology is",
    "values": {
      "-7": "No Answer",
      "1": "Very liberal",
      "2": "Liberal",
      "3": "Moderate",
      "4": "Conservative",
      "5": "Very conservative",
      "6": "Not sure"
    }
  },

    "child18": {
    "name": "Regarding children, you",
    "values": {
      "1": "Have children living at home",
      "2": "Do not have children living at home"
    }
  },
  "employ": {
    "name": "Your employment status is",
    "values": {
      "-7": "No Answer",
      "-1": "Inapplicable, legitimate skip",
      "1": "Full-time",
      "2": "Part-time",
      "3": "Temporarily laid off",
      "4": "Unemployed",
      "5": "Retired",
      "6": "Permanently disabled",
      "7": "Homemaker"
    }
  },
  "urbanicity2": {
    "name": "You live in a",
    "values": {
      "1": "Big city",
      "2": "Smaller city",
      "3": "Suburban area",
      "4": "Small town",
      "5": "Rural area"
    }
  },

  "inputstate":{
    "name": "You live in the state of",
    "values":{
      "10": "Delaware",
      "11": "District of Columbia",
      "12": "Florida",
      "13": "Georgia",
      "15": "Hawaii",
      "16": "Idaho",
      "17": "Illinois",
      "18": "Indiana",
      "19": "Iowa",
      "20": "Kansas",
      "21": "Kentucky",
      "22": "Louisiana",
      "23": "Maine",
      "24": "Maryland",
      "25": "Massachusetts",
      "26": "Michigan",
      "27": "Minnesota",
      "28": "Mississippi",
      "29": "Missouri",
      "30": "Montana",
      "31": "Nebraska",
      "32": "Nevada",
      "33": "New Hampshire",
      "34": "New Jersey",
      "35": "New Mexico",
      "36": "New York",
      "37": "North Carolina",
      "38": "North Dakota",
      "39": "Ohio",
      "40": "Oklahoma",
      "41": "Oregon",
      "42": "Pennsylvania",
      "44": "Rhode Island",
      "45": "South Carolina",
      "46": "South Dakota",
      "47": "Tennessee",
      "48": "Texas",
      "49": "Utah",
      "50": "Vermont",
      "51": "Virginia",
      "53": "Washington",
      "54": "West Virginia",
      "55": "Wisconsin",
      "56": "Wyoming",
      "01": "Alabama",
      "02": "Alaska",
      "04": "Arizona",
      "05": "Arkansas",
      "06": "California",
      "08": "Colorado",
      "09": "Connecticut",
      "72": "Puerto Rico",
      "66": "Guam",
      "78": "Virgin Islands",
      "60": "American Samoa"
    }
  }


}

def print_values(mapping):
    for key in mapping.keys():
        vals = mapping[key]['values']
        # Check if the values are nested dictionaries (e.g., for "race")
        if isinstance(vals, dict) and all(isinstance(v, dict) for v in vals.values()):
            for sub_key in vals.keys():
                sub_vals = vals[sub_key]
                # Sorting and printing each nested dictionary separately
                sorted_sub_vals = dict(sorted(sub_vals.items(), key=lambda item: int(item[0]) if item[0].isdigit() else item[0]))
                print(f"{mapping[key]['name']} ({sub_key}):")
                for val_key, val in sorted_sub_vals.items():
                    if val:  # Only print if there's a value to avoid empty strings
                        print(f"  {val_key}: {val}")
        else:
            # Sorting by key, converting to int if possible for proper numeric sorting
            sorted_vals = dict(sorted(vals.items(), key=lambda item: int(item[0]) if item[0].lstrip('-').isdigit() else item[0]))
            print(mapping[key]['name'] + ":")
            for val_key, val in sorted_vals.items():
                print(f"  {key}=={val_key}: {val}")
        print()  # Print a newline for better separation


class Agent:
    """A class to represent an agent that processes tasks based on specific characteristics.

    Attributes:
        model (str): The model version to use for processing.
        task_description (str): The description of the task to be processed.
        persona_prefix (str): The prefix to add before the persona description.
        persona_suffix (str): The suffix to add after the persona description.
        persona (str): The persona description to adopt for the task.
        system_instructions (str): The complete system instructions including persona and constraints.
        original_task_description (str): The original task description without modifications.
        current_task_description (str): The current task description that may include modifications.

    Args:
        task_description (str): The description of the task to be processed.
        data (pd.DataFrame, optional): The dataset to use for generating persona descriptions.
        persona_mapping (dict, optional): Mapping to convert dataset rows into persona descriptions.
        ideology (str, optional): The ideological filter to apply when selecting data for persona generation.
        query_str (str, optional): A string that is used for a pandas query clause on the dataframe
        model (str): The model version to use for processing. Defaults to "gpt-3.5-turbo".
        system_instructions (str): Additional instructions for the system. Defaults to an empty string.
        persona (str): Predefined persona description. If not provided, it will be generated based on data and ideology.
        persona_prefix (str): The prefix for the system instructions. Defaults to a standard instruction about adopting a persona.
        persona_suffix (str): The suffix for the system instructions. Defaults to standard constraints to follow.

    Methods:
        process_task(previous_response=""): Process the task, optionally building upon a previous response.
        get_persona_description(data, ideology): Generates a persona description based on the dataset and ideology.
        filter_data_by_ideology(data, ideology): Filters the dataset based on the specified ideology.
        _get_response(task): Internal method to interact with the API and get a response.
        row2persona(row, persona_mapping): Converts a dataset row into a persona description string.
    """
    def __init__(self,
                 task_description,
                 data=None,
                 persona_mapping = None,
                 ideology=None,
                 query_str = None,
                 model="gpt-3.5-turbo",
                 system_instructions="",
                 persona="",
                 persona_prefix="INSTRUCTIONS\nAdopt the following persona:\n\n",
                 persona_suffix="\n\nCONSTRAINTS\n- When answering, do not disclose your partisan or demographic identity in any way.\n- Think, talk, and write like your persona.\n- Use plain language.\n- Adopt the characteristics of your persona."):
        """Initialize an agent with specific characteristics and dataset."""
        self.model = model
        self.persona_mapping = persona_mapping
        self.task_description = task_description
        self.persona_prefix = persona_prefix
        self.persona_suffix = persona_suffix
        self.persona = persona
        self.ideology = ideology
        self.data = data
        self.query_str = query_str
        self.original_task_description = task_description # Original task description
        self.current_task_description = task_description # This will be updated when it's reading in from another agent
        self.validate()
        if not self.persona:
            if ideology:
              self.persona = self.get_persona_description_ideology(data, ideology)
            else:
              self.persona = self.row2persona(data.query(query_str).sample(1).iloc[0], self.persona_mapping)
        self.system_instructions = self.persona_prefix + self.persona + self.persona_suffix

    def process_task(self, previous_response=""):
        """Process the task, optionally building upon a previous response."""
        task = self.original_task_description
        if previous_response:
          task = f"""
{task}
INCORPORATE PRIOR ANSWERS
- Here is what was previously said: '''{previous_response}'''
- Do not respond directly to what was previously said, but keep the best points from what was previously said. Ensure the perspective from prior responses are represented in your balanced answer.
"""
        self.current_task_description = task
        print(self.current_task_description)
        return self._get_response(task)


    def get_persona_description_ideology(self, data, ideology):
        """Generates a persona description based on the dataset and ideology."""
        filtered_data = self.filter_data_by_ideology(data, ideology)
        if not filtered_data.empty:
            selected_row = filtered_data.sample(n=1).iloc[0]
            return self.row2persona(selected_row, self.persona_mapping)
        return "No data available for the specified ideology."

    def filter_data_by_ideology(self, data, ideology):
        """Filters the dataset based on the specified ideology."""
        if ideology == 'liberal':
            return data[data['ideo5'].isin([1, 2])]
        elif ideology == 'conservative':
            return data[data['ideo5'].isin([4, 5])]
        return data

    def _get_response(self, task):
        """Internal method to interact with the API and get a response."""
        try:
            completion = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": self.system_instructions},
                    {"role": "user", "content": task}
                ]
            )
            return completion.choices[0].message.content
        except Exception as e:
            print("Error in _get_response:", e)
            return None


    @staticmethod
    def row2persona(row, persona_mapping):
        """Converts a dataset row into a persona description string."""
        persona_description_str = ""
        for key, value in persona_mapping.items():
            field_name = value['name']
            if key == 'race':
              race_found = False
              for race_key in ['eth', 'rwh', 'rbl', 'rain', 'ras', 'rpi', 'roth']:
                  if race_key in row and str(row[race_key]) == '1':
                      race_description = value['values'][race_key].get('1', 'Unknown')
                      if race_description:  # If a description exists for the '1' value
                          persona_description_str += f"{field_name} {race_description}. "
                          race_found = True
                          break  # Stop checking once a race is found
              if not race_found:
                  # If no race was indicated or if the race data does not match expected values
                  persona_description_str += f"{field_name} Unknown. "
            elif key in row:
                if key == "birthyr_dropdown" and 'integer' in value['values']:
                    age = datetime.now().year - row[key]
                    persona_description_str += f"{field_name} {age}. "
                else:
                    mapped_value = value['values'].get(str(row[key]), 'Unknown')
                    if "inapplicable" not in mapped_value.lower() and "legitimate skip" not in mapped_value.lower() and "unknown" not in mapped_value.lower():
                      persona_description_str += f"{field_name} {mapped_value}. "
        return persona_description_str.lower()

    def validate(self):
      assert self.original_task_description is not None, "Need to provide some task instructions"
      if self.ideology or self.query_str:
        assert self.data is not None and self.persona_mapping is not None, "If you use either `ideology' or `query_str' you need to provide both a dataframe and a persona mapping to process rows of the dataframe."


class Chain:
    """A class to represent a chain of agents processing tasks in sequence.

    Attributes:
        agents (list): A list of Agent objects that will process the task.
        responses (list): A list of responses from each agent in the chain.
        prompts (list): A list of prompts used by each agent in the chain.
        final_response (str): The final response produced by the last agent in the chain.
        shuffle (bool): Whether to shuffle the order of agents in the chain.
        cycles (int): The number of cycles to process through the chain.
        last_n (int): The number of last responses to consider for the next task processing.

    Args:
        agents (list): A list of Agent objects that will process the task.
        shuffle (bool): If True, the order of agents will be shuffled. Defaults to False.
        cycles (int): The number of times the list of agents will process the task. Defaults to 1.
        last_n (int): The number of last responses to consider for the next task processing. Defaults to 1.

    Methods:
        process_chain(): Processes the task through the chain of agents, each building upon the last.
        format_prior_responses(responses): Formats prior responses for inclusion in the next task description.
    """
    def __init__(self, agents, shuffle=False, cycles=1, last_n=1):
        """Initialize a chain with a list of agents."""
        self.agents = agents
        self.responses = []
        self.prompts = []
        self.final_response = None
        self.shuffle = shuffle
        self.last_n = last_n
        if self.shuffle:
          self.agents = random.sample(self.agents, len(self.agents))
        self.cycles = 1
        if last_n == -9:
          self.last_n = 1000000


    def process_chain(self):
        """Process the task through a chain of agents, each building upon the last."""
        previous_responses = []
        for i in range(self.cycles):
          for i, agent in enumerate(self.agents):
              previous_responses_slice = previous_responses[:-self.last_n]
              previous_responses_str = self.format_previous_responses(previous_responses_slice)
              response = agent.process_task(self.format_previous_responses(previous_responses))
              print(response)
              previous_responses.append(response)
              self.prompts.append(agent.current_task_description)
              self.responses.append(response)
        self.final_response = self.responses[-1]

    @staticmethod
    def format_previous_responses(responses):
      if not responses:
        return ""
      else:
        resp_list = ["Response {}: {}".format(i, responses[i]) for i in range(len(responses))]
        resp_str = "\n".join(resp_list)
        return resp_str

# Assuming the dataset is loaded
data = pd.read_csv("https://raw.githubusercontent.com/josh-ashkinaze/plural_chains/main/anes_pilot_2022_csv_20221214.csv")

# Example of initializing agents with different ideologies
TASK = "How should the United States handle climate change? Answer in 50 words."
lib = Agent(data=data, ideology='liberal', task_description=TASK, persona_mapping=anes_mapping)
con = Agent(data=data, ideology='conservative', task_description=TASK, persona_mapping=anes_mapping)

lib_chain = Chain([lib, lib, lib])
con_chain = Chain([con, con, con])
mixed_chain = Chain([con, lib, con])

mixed_chain.process_chain()


<ipython-input-32-d97d6e9eb52d>:414: DtypeWarning: Columns (72) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("https://raw.githubusercontent.com/josh-ashkinaze/plural_chains/main/anes_pilot_2022_csv_20221214.csv")


How should the United States handle climate change? Answer in 50 words.
Error in _get_response: Error code: 401 - {'error': {'message': 'Incorrect API key provided: asd. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
None

How should the United States handle climate change? Answer in 50 words.
INCORPORATE PRIOR ANSWERS
- Here is what was previously said: '''Response 0: None'''
- Do not respond directly to what was previously said, but keep the best points from what was previously said. Ensure the perspective from prior responses are represented in your balanced answer.

Error in _get_response: Error code: 401 - {'error': {'message': 'Incorrect API key provided: asd. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
None

How should the United States handle climate change? Answer i

In [9]:
mixed_chain.responses

["I believe in taking care of our planet, but I also think it's important to balance environmental concerns with economic interests. We should focus on promoting innovation and technology to reduce emissions without hurting industries that provide jobs for families like mine.",
 "The United States needs to take strong action against climate change. We can invest in new technologies to cut down on pollution without harming jobs. It's crucial to strike a balance between safeguarding the environment and supporting industries that provide for American families like mine.",
 "I believe that we should address climate change by investing in innovative technologies that reduce emissions without harming industries that support American families. It's important to strike a balance between protecting our environment and preserving jobs. By promoting clean energy solutions, we can safeguard our planet without sacrificing economic opportunities for hardworking families like mine."]

In [15]:
anes_mapping['pid3']['values']

{'-7': 'No Answer',
 '1': 'Democrat',
 '2': 'Republican',
 '3': 'Independent',
 '4': 'Other',
 '5': 'Not sure'}

In [31]:
def print_values(mapping):
    for key in mapping.keys():
        vals = mapping[key]['values']
        # Check if the values are nested dictionaries (e.g., for "race")
        if isinstance(vals, dict) and all(isinstance(v, dict) for v in vals.values()):
            for sub_key in vals.keys():
                sub_vals = vals[sub_key]
                # Sorting and printing each nested dictionary separately
                sorted_sub_vals = dict(sorted(sub_vals.items(), key=lambda item: int(item[0]) if item[0].isdigit() else item[0]))
                print(f"{mapping[key]['name']} ({sub_key}):")
                for val_key, val in sorted_sub_vals.items():
                    if val:  # Only print if there's a value to avoid empty strings
                        print(f"  {val_key}: {val}")
        else:
            # Sorting by key, converting to int if possible for proper numeric sorting
            sorted_vals = dict(sorted(vals.items(), key=lambda item: int(item[0]) if item[0].lstrip('-').isdigit() else item[0]))
            print(mapping[key]['name'] + ":")
            for val_key, val in sorted_vals.items():
                print(f"  {key}=={val_key}: {val}")
        print()  # Print a newline for better separation

# Assuming anes_mapping is defined as provided, call the function
print_values(anes_mapping)


Your age is:
  birthyr_dropdown==integer: 

Your education is:
  educ==1: No high school
  educ==2: High school graduate
  educ==3: Some college
  educ==4: College graduate

Your gender is:
  gender==1: Male
  gender==2: Female

Your race is (eth):
  1: Hispanic
Your race is (rwh):
  1: White
Your race is (rbl):
  1: Black
Your race is (rain):
  1: American Indian or Alaska Native
Your race is (ras):
  1: Asian
Your race is (rpi):
  1: Native Hawaiian or Pacific Islander
Your race is (roth):
  1: Other

Politically, you identify as a(n):
  pid3==-7: No Answer
  pid3==1: Democrat
  pid3==2: Republican
  pid3==3: Independent
  pid3==4: Other
  pid3==5: Not sure

Your ideology is:
  ideo5==-7: No Answer
  ideo5==1: Very liberal
  ideo5==2: Liberal
  ideo5==3: Moderate
  ideo5==4: Conservative
  ideo5==5: Very conservative
  ideo5==6: Not sure

Regarding children, you:
  child18==1: Have children living at home
  child18==2: Do not have children living at home

Your employment status is:
 